In [20]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [4]:
df_flame = pd.read_csv("flame.txt", sep="\t")
df_jain = pd.read_csv("jain.txt", sep="\t")

In [8]:
column_names = ["x", "y", "type"]

df_flame.columns = column_names
df_jain.columns = column_names

In [9]:
df_flame.info

<bound method DataFrame.info of         x      y  type
0    1.35  26.65     1
1    1.40  23.25     2
2    0.85  23.05     2
3    0.50  22.35     2
4    0.65  21.35     2
..    ...    ...   ...
234  7.50  26.20     1
235  7.50  25.65     1
236  7.05  25.85     1
237  6.90  27.15     1
238  6.15  26.90     1

[239 rows x 3 columns]>

In [10]:
df_jain.info

<bound method DataFrame.info of          x      y  type
0     0.75  15.60     2
1     3.30  15.45     2
2     5.25  14.20     2
3     4.90  15.65     2
4     5.35  15.85     2
..     ...    ...   ...
367  38.75  16.85     1
368  39.00  16.60     1
369  38.25  17.35     1
370  39.50  16.95     1
371  39.90  17.05     1

[372 rows x 3 columns]>

In [22]:
for df in (df_flame, df_jain):
  X, y = df.iloc[:, 0:-1], df.iloc[:, -1]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

  